In [37]:
import django_initializer
import petl as etl
from asyncer import asyncify

import const

year_files = [
    (year, const.DATA_DIR / xlsx)
    for year, xlsx in [
        (1920, "Combined_PolkDirectory_1920.xlsx"),
        (1921, "Combined_PolkDirectory_1921.xlsx"),
        (1922, "Combined_PolkDirectory_1922.xlsx"),
    ]
]
year_files

[(1920,
  PosixPath('/app/greenwood_history/../data/Combined_PolkDirectory_1920.xlsx')),
 (1921,
  PosixPath('/app/greenwood_history/../data/Combined_PolkDirectory_1921.xlsx')),
 (1922,
  PosixPath('/app/greenwood_history/../data/Combined_PolkDirectory_1922.xlsx'))]

In [38]:
from asyncio import gather

constraints = [dict(name="street_name", field="street", test=str, optional=False)]

headers = {
    "original street": "original_street",
    "original NO": "original_number",
    "house number": "number",
    "street direction": "street_direction",
    "street name": "street",
    "street type": "street_type",
    "full address": "full_address",
    "gis object id": "gisid",
    "original 1920 city dir.": "original_dir",
    "salutation": "salutation",
    "suffix": "suffix",
    "last name": "last_name",
    "first name": "first_name",
    "profession/business type": "profession",
    "business name": "business",
    "not listed as african american in directory": "not_african_american",
    "Notes": "notes",
    "1920 census/head of residence": "",
    "listof losses (Parrish book)": "losses",
}


async def parse_file(y, file):
    table = await asyncify(etl.fromxlsx)(file)
    # table = etl.rename(table, headers)
    print(table)
    table = await asyncify(table.rename)(headers)
    # problems = await asyncify(etl.validate)(table, constraints=constraints)
    # print(problems.lookall())
    return y, table


all_data = await gather(*[parse_file(y, f) for y, f in year_files])

+-----------------+-------------+--------------+------------------+-------------+-------------+-----------------------+---------------+-------------------------+------------+--------+-----------+------------+--------------------------+------------------+---------------------------------------------+-------+-------------------------------+------------------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
| original street | original NO | house number | street direction | street name | street type | full address          | gis object id | original 1920 city dir. | salutation | suffix | last name | first name | profession/business type | business name    | not listed as african american in directory | Notes | 1920 census/head of residence | listof losses (Parrish book) | None | None | None | None | None | None | None | None | None | None | None | None | None | None | None |
+=================+=============+=============

In [31]:
from petl import dicts

for y, table in all_data:
    data = dicts(table)
    print(data)

{'original_street': 'Admiral Blvd.', 'original_number': '3', 'number': '3', 'street_direction': None, 'street': 'Admiral', 'street_type': 'Boulevard', 'full_address': '3 Admiral Boulevard', 'gisid': None, 'original_dir': 'address not listed', 'salutation': None, 'suffix': None, 'last_name': None, 'first_name': '', 'profession': '', 'business': None, 'not_african_american': None, 'notes': None, '': None, 'losses': None, 'None': None}
{'original_street': 'Admiral Blvd.', 'original_number': '800', 'number': '800', 'street_direction': None, 'street': 'Admiral', 'street_type': 'Boulevard', 'full_address': '800 Admiral Boulevard', 'gisid': None, 'original_dir': 'address not listed', 'salutation': None, 'suffix': None, 'last_name': None, 'first_name': '', 'profession': '', 'business': None, 'not_african_american': None, 'notes': None, '': 'Padilla, Victor (25) laborer', 'losses': None, 'None': None}
{'original_street': 'Admiral Blvd.', 'original_number': '801', 'number': '801', 'street_direct

FieldSelectionError: selection is not a field or valid field index: 'original 1920 city dir.'